# train

In [1]:
import yaml
import os

with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

/home/hiroki/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """


In [2]:
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
os.makedirs(OUTPUT_ROOT, exist_ok=True)

## load library

In [3]:
# python default library
import os
import shutil
import datetime
import sys
import pickle

# general analysis tool-kit
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# pytorch
import torch
from torch import optim, nn
from torch.utils.tensorboard import SummaryWriter

from torchsummary import summary

# etc
import yaml
yaml.warnings({'YAMLLoadWarning': False})
import mlflow
from collections import defaultdict
from scipy.stats import zscore

# original library

import common as com
import pytorch_modeler as modeler
from pytorch_model import DAGMM as Model
from pytorch_utils import filtered_load_model
import models

import librosa
import IPython
import librosa.display

## load config and set logger

In [4]:
#with open("./config.yaml", 'rb') as f:
#    config = yaml.load(f)

log_folder = config['IO_OPTION']['OUTPUT_ROOT']+'/{0}.log'.format(datetime.date.today())
logger = com.setup_logger(log_folder, '00_train.py')

/home/hiroki/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Setting

In [5]:
# Setting seed
modeler.set_seed(42)

In [6]:
############################################################################
# Setting I/O path
############################################################################
# input dirs
INPUT_ROOT = config['IO_OPTION']['INPUT_ROOT']
dev_path = INPUT_ROOT + "/dev_data"
add_dev_path = INPUT_ROOT + "/add_dev_data"
# machine type
MACHINE_TYPE = config['IO_OPTION']['MACHINE_TYPE']
machine_types = os.listdir(dev_path)
# output dirs
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
MODEL_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/models'
TB_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/tb'
PKL_DIR = OUTPUT_ROOT + '/pkl'
#os.makedirs(OUTPUT_ROOT, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(TB_DIR, exist_ok=True)
os.makedirs(PKL_DIR, exist_ok=True)
# copy config
shutil.copy('./config.yaml', OUTPUT_ROOT)

'/media/hiroki/working/research/dcase2020/result/2D/DAGMM_LSTM_CONV/config.yaml'

## make path list and train/valid split

In [7]:
############################################################################
# make path set and train/valid split
############################################################################
'''
train_paths[machine_type]['train' or 'valid'] = path
'''
dev_train_paths = {}
add_train_paths = {}
train_paths = {}

In [8]:
for machine_type in machine_types:
    # dev train
    dev_train_paths = ["{}/{}/train/".format(dev_path, machine_type) + file for file in os.listdir("{}/{}/train".format(dev_path, machine_type))]
    dev_train_paths = sorted(dev_train_paths)
    # add_dev train
    add_train_paths = ["{}/{}/train/".format(add_dev_path, machine_type) + file for file in os.listdir("{}/{}/train".format(add_dev_path, machine_type))]
    add_train_paths = sorted(add_train_paths)
    # valid
    dev_valid_paths = ["{}/{}/test/".format(dev_path, machine_type) + file for file in os.listdir("{}/{}/test".format(dev_path, machine_type))]
    dev_valid_paths = sorted(dev_valid_paths)
    
    train_paths[machine_type] = {}
    train_paths[machine_type]['train'] = dev_train_paths + add_train_paths
    train_paths[machine_type]['valid'] = dev_valid_paths

## training

In [9]:
#############################################################################
# run
#############################################################################
def run(machine_type):
    com.tic()
    logger.info('TARGET MACHINE_TYPE: {0}'.format(machine_type))
    logger.info('MAKE DATA_LOADER')
    # dev_train_paths
    dataloaders_dict = modeler.make_dataloader(train_paths, machine_type)
    # define writer for tensorbord
    os.makedirs(TB_DIR+'/'+machine_type, exist_ok=True)         # debug
    tb_log_dir = TB_DIR + '/' + machine_type
    writer = SummaryWriter(log_dir = tb_log_dir)
    logger.info('TRAINING')
    # parameter setting
    net = Model(sample_rate=config['preprocessing']['sample_rate'],
                window_size=config['preprocessing']['window_size'],
                hop_size=config['preprocessing']['hop_size'],
                mel_bins=config['preprocessing']['mel_bins'],
                fmin=config['preprocessing']['fmin'],
                fmax=config['preprocessing']['fmax'],
                latent_size=config['fit']['latent_size'],
                mixture_size=config['fit']['mixture_size'])
    #pretrained_dict = torch.load(config['IO_OPTION']['PREMODEL_PATH'])
    #net = filtered_load_model(net, pretrained_dict)
    optimizer = optim.Adam(net.parameters(), lr=1e-2)
    num_epochs = config['fit']['num_epochs']
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
                                              max_lr=1e-2, epochs=num_epochs, steps_per_epoch=len(dataloaders_dict['train']))
    criterion = nn.MSELoss()
    history = modeler.train_net(net, dataloaders_dict, criterion, optimizer, scheduler, num_epochs, writer)
    # output
    model = history['model']
    model_out_path = MODEL_DIR+'/{}_model.pth'.format(machine_type)
    torch.save(model.state_dict(), model_out_path)
    logger.info('\n success:{0} \n'.format(machine_type) + \
                    'model_out_path ==> \n {0}'.format(model_out_path))
    #  close writer for tensorbord
    writer.close()
    #modeler.mlflow_log(history, config, machine_type, model_out_path, tb_log_dir)
    com.toc()
    return history

In [10]:
machine_types

['fan', 'pump', 'slider', 'ToyCar', 'ToyConveyor', 'valve']

In [11]:
for machine_type in machine_types:
    history = run(machine_type)
    with open('{}/{}_history.pkl'.format(PKL_DIR, machine_type), 'wb') as file:
        pickle.dump(history , file)

2020-11-26 02:58:38,268 - 00_train.py - INFO - TARGET MACHINE_TYPE: fan
2020-11-26 02:58:38,268 - 00_train.py - INFO - MAKE DATA_LOADER
2020-11-26 02:58:39,132 - 00_train.py - INFO - TRAINING


use: cuda:0


100%|██████████| 29/29 [00:04<00:00,  6.12it/s]
2020-11-26 02:59:13,117 - pytorch_modeler.py - INFO - Epoch 1/100:train_loss:246.687064, tr_rec:10.025868, tr_eng:-45.362375, tr_covd:48239.487854, val_AUC:0.485466, val_pAUC:0.497065
100%|██████████| 29/29 [00:04<00:00,  6.10it/s]
2020-11-26 02:59:44,747 - pytorch_modeler.py - INFO - Epoch 2/100:train_loss:145.735523, tr_rec:7.820717, tr_eng:-191.221825, tr_covd:31407.398376, val_AUC:0.500584, val_pAUC:0.509950
100%|██████████| 29/29 [00:04<00:00,  6.20it/s]
2020-11-26 03:00:16,377 - pytorch_modeler.py - INFO - Epoch 3/100:train_loss:138.610774, tr_rec:7.068721, tr_eng:-195.453705, tr_covd:30217.485382, val_AUC:0.493170, val_pAUC:0.513669
100%|██████████| 29/29 [00:04<00:00,  6.15it/s]
2020-11-26 03:00:48,196 - pytorch_modeler.py - INFO - Epoch 4/100:train_loss:150.471278, tr_rec:7.713665, tr_eng:-209.367459, tr_covd:32738.872528, val_AUC:0.507173, val_pAUC:0.519854
100%|██████████| 29/29 [00:04<00:00,  5.82it/s]
2020-11-26 03:01:20,871 

100%|██████████| 29/29 [00:04<00:00,  6.14it/s]
2020-11-26 03:14:35,138 - pytorch_modeler.py - INFO - Epoch 30/100:train_loss:124.189845, tr_rec:10.827734, tr_eng:-265.759056, tr_covd:27987.603851, val_AUC:0.485266, val_pAUC:0.518072
100%|██████████| 29/29 [00:04<00:00,  6.24it/s]
2020-11-26 03:15:06,773 - pytorch_modeler.py - INFO - Epoch 31/100:train_loss:124.408128, tr_rec:10.749109, tr_eng:-263.762561, tr_covd:28007.055817, val_AUC:0.536273, val_pAUC:0.520070
100%|██████████| 29/29 [00:04<00:00,  6.31it/s]
2020-11-26 03:15:38,184 - pytorch_modeler.py - INFO - Epoch 32/100:train_loss:124.740398, tr_rec:11.056037, tr_eng:-264.742149, tr_covd:28031.715454, val_AUC:0.475074, val_pAUC:0.515277
100%|██████████| 29/29 [00:04<00:00,  6.16it/s]
2020-11-26 03:16:09,704 - pytorch_modeler.py - INFO - Epoch 33/100:train_loss:124.477942, tr_rec:10.929316, tr_eng:-267.258830, tr_covd:28054.902588, val_AUC:0.478272, val_pAUC:0.516084
100%|██████████| 29/29 [00:04<00:00,  6.25it/s]
2020-11-26 03:16

100%|██████████| 29/29 [00:04<00:00,  6.28it/s]
2020-11-26 03:29:49,016 - pytorch_modeler.py - INFO - Epoch 59/100:train_loss:123.714769, tr_rec:11.059618, tr_eng:-270.706822, tr_covd:27945.167450, val_AUC:0.524200, val_pAUC:0.496680
100%|██████████| 29/29 [00:04<00:00,  6.09it/s]
2020-11-26 03:30:20,600 - pytorch_modeler.py - INFO - Epoch 60/100:train_loss:124.491537, tr_rec:10.849630, tr_eng:-267.679007, tr_covd:28081.962250, val_AUC:0.512321, val_pAUC:0.521683
100%|██████████| 29/29 [00:04<00:00,  6.24it/s]
2020-11-26 03:30:52,178 - pytorch_modeler.py - INFO - Epoch 61/100:train_loss:124.597060, tr_rec:11.080001, tr_eng:-269.473511, tr_covd:28092.882721, val_AUC:0.484635, val_pAUC:0.495986
100%|██████████| 29/29 [00:04<00:00,  6.13it/s]
2020-11-26 03:31:23,909 - pytorch_modeler.py - INFO - Epoch 62/100:train_loss:123.910705, tr_rec:10.955513, tr_eng:-270.009776, tr_covd:27991.234375, val_AUC:0.525098, val_pAUC:0.516806
100%|██████████| 29/29 [00:04<00:00,  6.16it/s]
2020-11-26 03:31

...

100%|██████████| 14/14 [00:05<00:00,  2.42it/s]
2020-11-26 07:16:24,489 - pytorch_modeler.py - INFO - Epoch 1/100:train_loss:223.943626, tr_rec:8.968040, tr_eng:5.154408, tr_covd:42892.030304, val_AUC:0.481964, val_pAUC:0.492481
100%|██████████| 14/14 [00:02<00:00,  6.33it/s]
2020-11-26 07:16:50,525 - pytorch_modeler.py - INFO - Epoch 2/100:train_loss:132.411262, tr_rec:7.681891, tr_eng:-153.785790, tr_covd:28021.590576, val_AUC:0.477706, val_pAUC:0.494766
100%|██████████| 14/14 [00:02<00:00,  6.26it/s]
2020-11-26 07:17:16,601 - pytorch_modeler.py - INFO - Epoch 3/100:train_loss:121.968551, tr_rec:6.544302, tr_eng:-168.011671, tr_covd:26445.084015, val_AUC:0.480475, val_pAUC:0.497658
100%|██████████| 14/14 [00:02<00:00,  6.46it/s]
2020-11-26 07:17:42,477 - pytorch_modeler.py - INFO - Epoch 4/100:train_loss:119.892326, tr_rec:6.459129, tr_eng:-168.500036, tr_covd:26056.640747, val_AUC:0.476931, val_pAUC:0.494274
100%|██████████| 14/14 [00:02<00:00,  6.21it/s]
2020-11-26 07:18:08,606 - p

100%|██████████| 14/14 [00:02<00:00,  6.13it/s]
2020-11-26 07:29:03,308 - pytorch_modeler.py - INFO - Epoch 30/100:train_loss:111.257376, tr_rec:10.631783, tr_eng:-248.139523, tr_covd:25087.909637, val_AUC:0.495511, val_pAUC:0.508242
100%|██████████| 14/14 [00:02<00:00,  6.23it/s]
2020-11-26 07:29:29,611 - pytorch_modeler.py - INFO - Epoch 31/100:train_loss:111.040119, tr_rec:10.971681, tr_eng:-255.947991, tr_covd:25132.648041, val_AUC:0.503063, val_pAUC:0.507808
100%|██████████| 14/14 [00:02<00:00,  6.45it/s]
2020-11-26 07:29:55,635 - pytorch_modeler.py - INFO - Epoch 32/100:train_loss:111.178239, tr_rec:11.188626, tr_eng:-260.744028, tr_covd:25212.803772, val_AUC:0.468981, val_pAUC:0.497918
100%|██████████| 14/14 [00:02<00:00,  6.14it/s]
2020-11-26 07:30:21,940 - pytorch_modeler.py - INFO - Epoch 33/100:train_loss:110.635747, tr_rec:10.494210, tr_eng:-244.418634, tr_covd:24916.680664, val_AUC:0.492533, val_pAUC:0.503412
100%|██████████| 14/14 [00:02<00:00,  6.22it/s]
2020-11-26 07:30

100%|██████████| 14/14 [00:02<00:00,  6.21it/s]
2020-11-26 07:41:41,490 - pytorch_modeler.py - INFO - Epoch 59/100:train_loss:110.098536, tr_rec:10.171651, tr_eng:-241.913424, tr_covd:24823.646057, val_AUC:0.476277, val_pAUC:0.497687
100%|██████████| 14/14 [00:02<00:00,  6.01it/s]
2020-11-26 07:42:07,799 - pytorch_modeler.py - INFO - Epoch 60/100:train_loss:110.415774, tr_rec:9.872158, tr_eng:-235.925549, tr_covd:24827.234619, val_AUC:0.488824, val_pAUC:0.495200
100%|██████████| 14/14 [00:02<00:00,  6.16it/s]
2020-11-26 07:42:34,043 - pytorch_modeler.py - INFO - Epoch 61/100:train_loss:110.130904, tr_rec:9.994329, tr_eng:-241.191089, tr_covd:24851.137451, val_AUC:0.493115, val_pAUC:0.493638
100%|██████████| 14/14 [00:02<00:00,  6.18it/s]
2020-11-26 07:43:00,271 - pytorch_modeler.py - INFO - Epoch 62/100:train_loss:109.911243, tr_rec:10.167409, tr_eng:-246.109883, tr_covd:24870.965057, val_AUC:0.486893, val_pAUC:0.502429
100%|██████████| 14/14 [00:02<00:00,  6.27it/s]
2020-11-26 07:43:2

100%|██████████| 14/14 [00:02<00:00,  6.23it/s]
2020-11-26 07:54:20,298 - pytorch_modeler.py - INFO - Epoch 88/100:train_loss:109.442227, tr_rec:10.088185, tr_eng:-245.829868, tr_covd:24787.406555, val_AUC:0.493893, val_pAUC:0.503036
100%|██████████| 14/14 [00:02<00:00,  6.32it/s]
2020-11-26 07:54:46,483 - pytorch_modeler.py - INFO - Epoch 89/100:train_loss:109.564363, tr_rec:10.085084, tr_eng:-245.322750, tr_covd:24802.311646, val_AUC:0.486137, val_pAUC:0.498612
100%|██████████| 14/14 [00:02<00:00,  6.21it/s]
2020-11-26 07:55:12,613 - pytorch_modeler.py - INFO - Epoch 90/100:train_loss:109.563422, tr_rec:10.105717, tr_eng:-245.558845, tr_covd:24802.718506, val_AUC:0.500912, val_pAUC:0.503065
100%|██████████| 14/14 [00:02<00:00,  6.35it/s]
2020-11-26 07:55:38,640 - pytorch_modeler.py - INFO - Epoch 91/100:train_loss:109.508630, tr_rec:10.039349, tr_eng:-245.115566, tr_covd:24796.168060, val_AUC:0.503736, val_pAUC:0.500868
100%|██████████| 14/14 [00:02<00:00,  6.34it/s]
2020-11-26 07:56

elapsed time: 2695.578222036 [sec]


history = run('ToyCar')
with open('{}/{}_history.pkl'.format(PKL_DIR, machine_type), 'wb') as file:
    pickle.dump(history , file)